In [1]:
import numpy as np
import pandas as pd
from pycaret.regression import *
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
model_lgbm_0=load_model("models/lgbm_v0")
model_lgbm_1=load_model("models/lgbm_v1")
model_lasso_0=load_model("models/lasso_v0")
#model_lasso_1=load_model("models/lasso_v1")

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [3]:
df_material=pd.DataFrame([['AISI_1045', 0.25, 1000, -1, 5, 209.3996337890626, 553.0, 600.0,0.0134, 0.234, 1.0],
                          ['AISI_4140', 0.25, 1000, -4, 5, 1147.44189453125, 595.0, 580.0,0.023, 0.133, 1.03],
                          ['AISI_4340', 0.25, 1000, -4, 5, 584.8319091796877, 792.0, 510.0,0.014, 1.03, 0.26],
                          ['AISI_52100', 0.25, 1000, -4, 5, 253.8834747314454, 774.78, 134.0,0.0018, 0.37, 3.171],
                          ['AISI_304', 0.25, 1000, -4, 5, 739.4136962890625, 310.0, 1000.0,0.07, 0.65, 1.0],
                          ['AISI_1020', 0.25, 1000, -4, 5, 80.56342620849611, 213.0, 53.0,0.0055, 0.34, 0.81],],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m"])
df_material

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m
0,AISI_1045,0.25,1000,-1,5,209.399634,553.00,600.0,0.0134,0.234,1.000
1,AISI_4140,0.25,1000,-4,5,1147.441895,595.00,580.0,0.0230,0.133,1.030
2,AISI_4340,0.25,1000,-4,5,584.831909,792.00,510.0,0.0140,1.030,0.260
3,AISI_52100,0.25,1000,-4,5,253.883475,774.78,134.0,0.0018,0.370,3.171
4,AISI_304,0.25,1000,-4,5,739.413696,310.00,1000.0,0.0700,0.650,1.000
5,AISI_1020,0.25,1000,-4,5,80.563426,213.00,53.0,0.0055,0.340,0.810


### Teste de otimização com GA para um modelo de machine learning

Para esse teste vamos fixar os parâmetros referentes ao material e só iremos otimizar o parâmetros depth,vc,rake_angle.

In [4]:
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [5]:
model_lasso_0.named_steps["trained_model"].__class__.__name__

'Lasso'

In [6]:
model_lgbm_0.named_steps["trained_model"].__class__.__name__

'LGBMRegressor'

In [7]:
def objective_function_with_penalties(model,inputs):
    #ToDo: construir uma função que replique a Maximiação do MMR com restrição na potência conforme feito para a otimização teórica.
    pass

In [9]:
import plotly.graph_objects as go

x_min, x_max = -10, 10
y_min, y_max = 500, 5000

x_values = np.linspace(x_min, x_max, 100)
y_values = np.linspace(y_min, y_max, 100)
X_grid, Y_grid = np.meshgrid(x_values, y_values)

input_data = np.column_stack((X_grid.ravel(), Y_grid.ravel()))
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int","neg_allowed":True},
                "vc":{"sequence_len":15,"max_bound":10000,"lower_bound":500,"type":"int","neg_allowed":False},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

final_df=pd.DataFrame()
for material_index,material_name in enumerate(df_material["material"].unique()):
    for model_artifact in [model_lasso_0,model_lgbm_0]:
        model_name=model_artifact.named_steps["trained_model"].__class__.__name__
        print(material_name, model_name)
        pd_inputs=build_input(input_data,df_material.iloc[[material_index]],construct_dict)
        predictions = predict_model(model_artifact,pd_inputs.reset_index(drop=True))
        
        final_df=pd.concat([predictions.assign(model=model_name),final_df])
final_df.sort_values("prediction_label").drop_duplicates(["material","model"],keep="first").sort_values(["material","model"])[
    ["model","material","vc","rake_angle","prediction_label"]
]

AISI_1045 Lasso
AISI_1045 LGBMRegressor
AISI_4140 Lasso
AISI_4140 LGBMRegressor
AISI_4340 Lasso
AISI_4340 LGBMRegressor
AISI_52100 Lasso
AISI_52100 LGBMRegressor
AISI_304 Lasso
AISI_304 LGBMRegressor
AISI_1020 Lasso
AISI_1020 LGBMRegressor


,model,material,vc,rake_angle,prediction_label
8291,LGBMRegressor,AISI_1020,4227.272949,8.383839,86.170629
67,Lasso,AISI_1020,500.000000,3.535353,62.278825
9608,LGBMRegressor,AISI_1045,4863.636230,-8.383839,205.321183
46,Lasso,AISI_1045,500.000000,-0.707071,213.005705
3099,LGBMRegressor,AISI_304,1863.636353,10.000000,337.053043
99,Lasso,AISI_304,500.000000,10.000000,656.229864
9690,LGBMRegressor,AISI_4140,4863.636230,8.181818,238.466585
34,Lasso,AISI_4140,500.000000,-3.131313,1121.418123
4761,LGBMRegressor,AISI_4340,2636.363525,2.323232,573.275700
0,Lasso,AISI_4340,500.000000,-10.000000,545.145623
